<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook demonstrates the advances concepts of fairness detection using Indirect Bias mechanism using IBM Watson OpenScale.

We make use of the adult income dataset, where the attributes `age`, `sex` and `race` are not used to train the machine learning model that we are building as part of this notebook. But we will configure fairness against these attributes (specifically `age` and `sex`) to check if the model is indirectly behaving in a biased manner with these protected (sensitive) attributes.

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale and an instance of IBM Watson Machine Learning using the Cloud catalog.

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, click your name, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [Instructions](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

<b>How to get api key using console:</b>

<li> bx login --sso
<li> bx iam api-key-create 'my_key'

## Credentials for IBM Cloud services

### Retrieve your IBM Cloud API key

1.	From the IBM Cloud toolbar, click your Account name, such as <Your user name>’s Account.
1.	From the Manage menu, click Access (IAM).
1.	In the navigation bar, click IBM Cloud API keys.
1.	Click the Create an IBM Cloud API key button.
1.	Type a name and description and then click Save.
1.	Copy the newly created API key and paste it into your notebook in the following **CLOUD_API_KEY** code box, which is the first code box.

    Note: replace everything between the two sets of double quotation marks (").

In [1]:
CLOUD_API_KEY = "<Your USER API KEY>"

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200719111718-0001
KERNEL_ID = f622af09-8284-4f0c-b685-2bfe77df32c1


### Retrieve your Watson Machine Learning credentials

1.	Go to the IBM Cloud dashboard.
1.	In the Resource summary section, click Services.
1.	Click Machine Learning.
1.	In the navigation pane, click Service credentials.
1.	Click the New credential button.
1.	Copy your credentials by clicking the copy icon.
1.	Return to the notebook editor and update the credentials by replacing the sample credentials with your own in the below cell

   **Note**: You need to replace everything including the opening bracket ({) and the closing bracket (}).
   
   **IMPORTANT**: If you are reusing a WML instance that is already bound to Watson OpenScale. Please specify that instance credentials in `wml_credentials`

In [2]:
# Enter wml credentials
wml_credentials = {
  "apikey": "CJqmD67Vtk7pw8U_rGicp82OV1vmdbpkMZRUbNXmEwwk",
  "iam_apikey_description": "Auto-generated for key 7896b158-8f2a-4b8d-8db6-b0797bbaa323",
  "iam_apikey_name": "Service credentials-2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e40741b27da5881193d18b40e6a3078d::serviceid:ServiceId-bd2d3a5d-61fc-4522-a2ac-3f6f6d482fbf",
  "instance_id": "8941e786-01c8-428f-ba1a-0221993fa9a8",
  "url": "https://us-south.ml.cloud.ibm.com"
}


In [3]:
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

# Package installation

The following opensource packages must be installed into this notebook instance so that they are available to use during processing.

In [4]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install psycopg2-binary | tail -n 1
!pip install pyspark==2.3.0

tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
watson-machine-learning-client-v4 1.0.95 has requirement ibm-cos-sdk==2.6.0, but you'll have ibm-cos-sdk 2.6.3 which is incompatible.
watson-machine-learning-client-v4 1.0.95 has requirement pandas<=0.25.3, but you'll have pandas 1.0.5 which is incompatible.
pytest-openfiles 0.5.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.
pytest-doctestplus 0.7.0 has requirement pytest>=4.0, but you'll have pytest 3.10.1 which is incompatible.
pytest-astropy 0.8.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.9 which is incompatible.
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
py

Restart the kernel to assure the new libraries are being used.

# Load and explore data

In [5]:
!rm adult.csv
!wget https://raw.githubusercontent.com/ravichamarthy/indirect-bias/master/adult.csv

rm: cannot remove ‘adult.csv’: No such file or directory
--2020-07-19 11:19:35--  https://raw.githubusercontent.com/ravichamarthy/indirect-bias/master/adult.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3551145 (3.4M) [text/plain]
Saving to: ‘adult.csv’

100%[======================================>] 3,551,145   --.-K/s   in 0.07s   

2020-07-19 11:19:35 (48.9 MB/s) - ‘adult.csv’ saved [3551145/3551145]



## Explore data

In [6]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="adult.csv", sep=",", header=True, inferSchema=True) 
df_data.head()

Row(age=39, workclass='State-gov', fnlwgt=77516, education='Bachelors', education-num=13, Marital='Never-married', occupation='Adm-clerical', relationship='Not-in-family', race='White', sex='Male', capitalgain=2174, loss=0, hoursper=40, citizen_status='United-States', label='<=50K')

In [7]:
print("Number of records: " + str(df_data.count()))

Number of records: 32561


# Create a model

In [8]:
# spark_df = sqlCtx.createDataFrame(df_data)
spark_df = df_data
# Remove protected attributes from training data
protected_attributes = ["race", "age", "sex"]
for attr in protected_attributes:
    spark_df = spark_df.drop(attr)
columns = spark_df.columns
model_name = "Income Classifier Binary Model"
deployment_name = "Income Classifier Binary Deployment"

spark_df.printSchema()

root
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- Marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- capitalgain: integer (nullable = true)
 |-- loss: integer (nullable = true)
 |-- hoursper: integer (nullable = true)
 |-- citizen_status: string (nullable = true)
 |-- label: string (nullable = true)



In [9]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml import Pipeline, Model

cat_features = ['workclass', 'education', 'Marital', 'occupation', 'relationship', 'citizen_status'] 
num_features = ["fnlwgt", "education-num", "capitalgain", "loss", "hoursper"]
stages=[]

for feature in cat_features:
    string_indexer = StringIndexer(inputCol = feature, outputCol = feature + '_IX').setHandleInvalid("keep")
    encoder = OneHotEncoderEstimator(inputCols=[string_indexer.getOutputCol()], outputCols=[feature + "classVec"])
    stages += [string_indexer, encoder]

si_Label = StringIndexer(inputCol="label", outputCol="encoded_label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)
stages.append(si_Label)

In [10]:
assembler_inputs = [c + "classVec" for c in cat_features] + num_features
va_features = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages.append(va_features)
va_features

VectorAssembler_4bcabacad453d869ddea

In [11]:
assembler_inputs

['workclassclassVec',
 'educationclassVec',
 'MaritalclassVec',
 'occupationclassVec',
 'relationshipclassVec',
 'citizen_statusclassVec',
 'fnlwgt',
 'education-num',
 'capitalgain',
 'loss',
 'hoursper']

In [12]:
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

Number of records for training: 26028
Number of records for evaluation: 6533


In [13]:
train_data.columns

['workclass',
 'fnlwgt',
 'education',
 'education-num',
 'Marital',
 'occupation',
 'relationship',
 'capitalgain',
 'loss',
 'hoursper',
 'citizen_status',
 'label']

In [14]:
from pyspark.ml.classification import GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
classifier = RandomForestClassifier(labelCol="encoded_label", featuresCol="features")
stages.append(classifier)
stages.append(label_converter)
print(stages)
pipeline = Pipeline(stages=stages)
model = pipeline.fit(train_data)

[StringIndexer_418fbbf86833fb289e0d, OneHotEncoderEstimator_4c9da481a23a4a2ebff3, StringIndexer_4ae8a0a3a95d7b2b6816, OneHotEncoderEstimator_4cb6950bcd568b2e0766, StringIndexer_4f95bbcadca23c0c2acf, OneHotEncoderEstimator_469fb20d343583eca8af, StringIndexer_4771bbe1bd4b43ec9e9a, OneHotEncoderEstimator_40eca18d80f66ad94979, StringIndexer_46b68e9336ba3815ea49, OneHotEncoderEstimator_47618cfdcc80d1755f05, StringIndexer_42658459d1dad51c19db, OneHotEncoderEstimator_46cdb78cea4ebbe74a3e, StringIndexer_4e758c3350b7b052fa63, VectorAssembler_4bcabacad453d869ddea, RandomForestClassifier_43efa8cfd8bdc8f91657, IndexToString_4b09b18fe23caf7cd04a]


In [15]:
predictions = model.transform(test_data)
predictions.printSchema()
predictions.head()

root
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- Marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- capitalgain: integer (nullable = true)
 |-- loss: integer (nullable = true)
 |-- hoursper: integer (nullable = true)
 |-- citizen_status: string (nullable = true)
 |-- label: string (nullable = true)
 |-- workclass_IX: double (nullable = false)
 |-- workclassclassVec: vector (nullable = true)
 |-- education_IX: double (nullable = false)
 |-- educationclassVec: vector (nullable = true)
 |-- Marital_IX: double (nullable = false)
 |-- MaritalclassVec: vector (nullable = true)
 |-- occupation_IX: double (nullable = false)
 |-- occupationclassVec: vector (nullable = true)
 |-- relationship_IX: double (nullable = false)
 |-- relationshipclassVec: vector (nullable = true)
 |-- citizen_status_IX: 

Row(workclass='?', fnlwgt=12285, education='Some-college', education-num=10, Marital='Never-married', occupation='?', relationship='Not-in-family', capitalgain=0, loss=0, hoursper=20, citizen_status='United-States', label='<=50K', workclass_IX=3.0, workclassclassVec=SparseVector(9, {3: 1.0}), education_IX=1.0, educationclassVec=SparseVector(16, {1: 1.0}), Marital_IX=1.0, MaritalclassVec=SparseVector(7, {1: 1.0}), occupation_IX=7.0, occupationclassVec=SparseVector(15, {7: 1.0}), relationship_IX=1.0, relationshipclassVec=SparseVector(6, {1: 1.0}), citizen_status_IX=0.0, citizen_statusclassVec=SparseVector(41, {0: 1.0}), encoded_label=0.0, features=SparseVector(99, {3: 1.0, 10: 1.0, 26: 1.0, 39: 1.0, 48: 1.0, 53: 1.0, 94: 12285.0, 95: 10.0, 98: 20.0}), rawPrediction=DenseVector([18.8473, 1.1527]), probability=DenseVector([0.9424, 0.0576]), prediction=0.0, predictedLabel='<=50K')

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluatorDT = BinaryClassificationEvaluator(labelCol="encoded_label", rawPredictionCol="rawPrediction")
accuracy = evaluatorDT.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.881775


# Save and deploy the model

In [17]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [18]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(model_name),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.8
        }
    ]
}

In [19]:
model_uid = None
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == deployment_name:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid)

print("Storing model ...")

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")

Deleting existing deployment with id 201cc957-6967-4df0-ac71-f239f80d0b44
Deleting existing model with id 0f56e702-23d6-4b6b-bd01-caefb716bc4c
Storing model ...
Done


In [20]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if deployment_name == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=deployment_name, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '1f426b8c-1a05-47c6-97ef-3c45d9e460a7' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='85fa8a9a-63d1-4bf2-831b-d0db19733a7f'
------------------------------------------------------------------------------------------------


Model id: 1f426b8c-1a05-47c6-97ef-3c45d9e460a7
Deployment id: 85fa8a9a-63d1-4bf2-831b-d0db19733a7f


In [21]:
import pandas as pd

df = pd.read_csv("adult.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,Marital,occupation,relationship,race,sex,capitalgain,loss,hoursper,citizen_status,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [22]:
cols_to_remove = ['label']
cols_to_remove.extend(protected_attributes)
cols_to_remove

['label', 'race', 'age', 'sex']

## Create the meta data frame capturing the sensitive data

In [23]:
meta_df = df[protected_attributes].copy()
meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

## Construct the scoring payload comprising the meta fields

In [24]:
def get_scoring_payload(no_of_records_to_score = 1):
    meta_payload = {
        "fields": meta_fields,
        "values": meta_values[:no_of_records_to_score]
    }

    for col in cols_to_remove:
        if col in df.columns:
            del df[col] 

    fields = df.columns.tolist()
    values = df[fields].values.tolist()

    payload_scoring = {"fields": fields,"values": values}

    payload_scoring = {
        "fields": fields,
        "values": values[:no_of_records_to_score],
        "meta": meta_payload
    }

    #import json
    #print(json.dumps(payload_scoring, indent=None))    
    return payload_scoring

## Method to perform scoring

In [25]:
def payload_logging(no_of_records_to_score = 1):
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    scoring_endpoint = None

    for deployment in wml_client.deployments.get_details()['resources']:
        if deployment_uid in deployment['metadata']['guid']:
            scoring_endpoint = deployment['entity']['scoring_url']

    print(scoring_endpoint)    
    scoring_response = wml_client.deployments.score(scoring_endpoint, payload_scoring)
    
    print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])
    #print(json.dumps(scoring_response, indent=None))

## Score the model and print the scoring response

In [26]:
payload_logging(no_of_records_to_score = 1)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/8941e786-01c8-428f-ba1a-0221993fa9a8/deployments/85fa8a9a-63d1-4bf2-831b-d0db19733a7f/online
Single record scoring result: 
 fields: ['workclass', 'fnlwgt', 'education', 'education-num', 'Marital', 'occupation', 'relationship', 'capitalgain', 'loss', 'hoursper', 'citizen_status', 'workclass_IX', 'workclassclassVec', 'education_IX', 'educationclassVec', 'Marital_IX', 'MaritalclassVec', 'occupation_IX', 'occupationclassVec', 'relationship_IX', 'relationshipclassVec', 'citizen_status_IX', 'citizen_statusclassVec', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 2174, 0, 40, 'United-States', 4.0, [9, [4], [1.0]], 2.0, [16, [2], [1.0]], 1.0, [7, [1], [1.0]], 3.0, [15, [3], [1.0]], 1.0, [6, [1], [1.0]], 0.0, [41, [0], [1.0]], [99, [4, 11, 26, 35, 48, 53, 94, 95, 96, 98], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 77516.0, 13.0, 

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [27]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.
1. Please update the `url` in the below WOS_CREDENTIALS payload as per the environment that you are using.
2. Please update the `DASHBOARD_URL` in the below cell as per the environment that you are using.

In [28]:
from ibm_ai_openscale.utils import get_instance_guid


WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.test.cloud.ibm.com"
}
DASHBOARD_URL = "https://aiopenscale.test.cloud.ibm.com"


if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

68f38b69-fe5e-4bd1-9c8c-a475038661f6


In [29]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

'2.2.1'

## Generate an IAM token
### The following is a function that will generate an IAM access token used to interact with the Watson OpenScale APIs

In [30]:
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth("bx", "bx")
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY
    }
    response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the Income classifier model will be removed from OpenScale monitoring.

In [31]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])

Using existing internal datamart.


In [32]:
data_mart_details = ai_client.data_mart.get_details()

In [33]:
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'internal_database_pool': 'icd-psql',
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/e40741b27da5881193d18b40e6a3078d:68f38b69-fe5e-4bd1-9c8c-a475038661f6::',
 'status': {'state': 'active'}}

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Pre-Prod" already exists, this code will delete that binding a create a new one.

In [34]:
all_bindings = ai_client.data_mart.bindings.get_details()['service_bindings']
existing_binding = False
for binding in all_bindings:
    binding_uid = binding['metadata']['guid']
    if binding['metadata']['guid'] == wml_credentials['instance_id']:
        existing_binding = True
        break

if not existing_binding:
    binding_uid = ai_client.data_mart.bindings.add('WML Prod', WatsonMachineLearningInstance(wml_credentials))
    
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

8941e786-01c8-428f-ba1a-0221993fa9a8,WML Prod,watson_machine_learning,2020-07-18T17:55:51.230Z


In [35]:
print(binding_uid)
ai_client.data_mart.bindings.get_details(binding_uid)

8941e786-01c8-428f-ba1a-0221993fa9a8


{'entity': {'credentials': {'apikey': 'CJqmD67Vtk7pw8U_rGicp82OV1vmdbpkMZRUbNXmEwwk',
   'iam_apikey_description': 'Auto-generated for key 7896b158-8f2a-4b8d-8db6-b0797bbaa323',
   'iam_apikey_name': 'Service credentials-2',
   'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Manager',
   'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/e40741b27da5881193d18b40e6a3078d::serviceid:ServiceId-bd2d3a5d-61fc-4522-a2ac-3f6f6d482fbf',
   'instance_id': '8941e786-01c8-428f-ba1a-0221993fa9a8',
   'url': 'https://us-south.ml.cloud.ibm.com'},
  'instance_id': '8941e786-01c8-428f-ba1a-0221993fa9a8',
  'name': 'WML Prod',
  'operational_space_id': 'production',
  'service_type': 'watson_machine_learning',
  'status': {'state': 'active'}},
 'metadata': {'guid': '8941e786-01c8-428f-ba1a-0221993fa9a8',
  'url': '/v1/data_marts/68f38b69-fe5e-4bd1-9c8c-a475038661f6/service_bindings/8941e786-01c8-428f-ba1a-0221993fa9a8',
  'created_at': '2020-07-18T17:55:51.230Z',
  'modified_at': '20

In [36]:
ai_client.data_mart.bindings.list_assets()

1f426b8c-1a05-47c6-97ef-3c45d9e460a7,Income Classifier Binary Model,2020-07-19T11:21:42.517Z,model,mllib-2.3,8941e786-01c8-428f-ba1a-0221993fa9a8,False
a012bbf0-b415-4b38-a5fc-923ac3394389,GermanCreditRiskModelYPQA,2020-07-16T18:33:48.316Z,model,mllib-2.3,8941e786-01c8-428f-ba1a-0221993fa9a8,False


## Patch binding as production

In [37]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = [
 {
   "op": "replace",
   "path": "/operational_space_id",
   "value": "production"
 }
]

In [38]:
SERVICE_PROVIDER_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/service_providers/{1}".format(WOS_GUID, binding_uid)
response = requests.patch(SERVICE_PROVIDER_URL, json=payload, headers=headers)
json_data = response.json()
print(json_data)

{'entity': {'credentials': {'secret_id': '75e40a81-1603-4d9c-8246-3fed5e5b997e'}, 'instance_id': '8941e786-01c8-428f-ba1a-0221993fa9a8', 'name': 'WML Prod', 'operational_space_id': 'production', 'service_type': 'watson_machine_learning', 'status': {'state': 'active'}}, 'metadata': {'created_at': '2020-07-18T17:55:51.230Z', 'created_by': 'IBMid-270007D500', 'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/e40741b27da5881193d18b40e6a3078d:68f38b69-fe5e-4bd1-9c8c-a475038661f6:service_provider:8941e786-01c8-428f-ba1a-0221993fa9a8', 'id': '8941e786-01c8-428f-ba1a-0221993fa9a8', 'modified_at': '2020-07-19T11:22:02.572Z', 'modified_by': 'IBMid-270007D500', 'url': '/v2/service_providers/8941e786-01c8-428f-ba1a-0221993fa9a8'}}


## Remove existing prod subscription

### This code removes previous subscription that matches the name German Credit Risk Model - Prod as it is expected this subscription is created only via this notebook.

In [39]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == model_name:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', sub_name)

Deleted existing subscription for Income Classifier Binary Model


In [40]:
feature_columns = []
feature_columns.extend(cat_features)
feature_columns.extend(num_features)
feature_columns

['workclass',
 'education',
 'Marital',
 'occupation',
 'relationship',
 'citizen_status',
 'fnlwgt',
 'education-num',
 'capitalgain',
 'loss',
 'hoursper']

In [41]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    binding_uid=binding_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='label',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = feature_columns,
    categorical_columns = cat_features
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == model_name:
            subscription = ai_client.data_mart.subscriptions.get(sub)

In [42]:
ai_client.data_mart.subscriptions.list()

e900311f-0f30-47c9-8973-19092e028555,Income Classifier Binary Model,model,8941e786-01c8-428f-ba1a-0221993fa9a8,2020-07-19T11:22:07.101Z


In [43]:
subscription.uid

'e900311f-0f30-47c9-8973-19092e028555'

## Patch the training data reference to the challenger subscription

In [44]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

# training_data_reference = {
#   "connection": {
#     "database_name": "BLUDB",
#     "hostname": "dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net",
#     "port": "50001",
#     "password": "dwrhzsgtvv9cf3-z",
#     "username": "nmx87075",
#     "ssl": True
#   },
#   "location": {
#     "schema_name": "NMX87075",
#     "table_name": "ADULT_INCOME"
#   },
#   "type": "db2"
# }

training_data_reference = {
                "connection": {
                    "api_key": "Y2O5Crx7jnFK5umKXRI-L9vxMDhjiv1o6ZdjfBvuMRZD",
                    "iam_url": "https://iam.cloud.ibm.com/oidc/token",
                    "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/e40741b27da5881193d18b40e6a3078d:30030db1-808f-4a80-8f70-2a85ce8948b8::",
                    "url": "https://s3.us.cloud-object-storage.appdomain.cloud"
                },
                "location": {
                    "bucket": "aif360experiments-donotdelete-pr-bsv6puguiwtb5b",
                    "file_format": "csv",
                    "file_name": "adult.csv",
                    "firstlineheader": True,
                    "infer_schema": "1"
                },
                "type": "cos"
            }

payload = [
 {
   "op": "replace",
   "path": "/asset_properties/training_data_reference",
   "value": training_data_reference
 }
]

In [45]:
SUBSCRIPTION_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}/subscriptions/{2}".format(WOS_GUID, binding_uid, subscription.uid)

response = requests.patch(SUBSCRIPTION_URL, json=payload, headers=headers)
json_data = response.json()
print(json_data)

{'entity': {'asset': {'asset_id': '1f426b8c-1a05-47c6-97ef-3c45d9e460a7', 'asset_type': 'model', 'created_at': '2020-07-19T11:21:42.517Z', 'name': 'Income Classifier Binary Model', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/8941e786-01c8-428f-ba1a-0221993fa9a8/published_models/1f426b8c-1a05-47c6-97ef-3c45d9e460a7'}, 'asset_properties': {'categorical_fields': ['workclass', 'education', 'Marital', 'occupation', 'relationship', 'citizen_status'], 'feature_fields': ['workclass', 'education', 'Marital', 'occupation', 'relationship', 'citizen_status', 'fnlwgt', 'education-num', 'capitalgain', 'loss', 'hoursper'], 'input_data_schema': {'fields': [{'metadata': {'measure': 'discrete', 'modeling_role': 'feature'}, 'name': 'workclass', 'nullable': True, 'type': 'string'}, {'metadata': {'modeling_role': 'feature'}, 'name': 'fnlwgt', 'nullable': True, 'type': 'integer'}, {'metadata': {'measure': 'discrete', 'modeling_role': 'feature'}, 'name': 'education', 'nullable': True, 'type': 

## Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [46]:
payload_logging(no_of_records_to_score = 200)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/8941e786-01c8-428f-ba1a-0221993fa9a8/deployments/85fa8a9a-63d1-4bf2-831b-d0db19733a7f/online
Single record scoring result: 
 fields: ['workclass', 'fnlwgt', 'education', 'education-num', 'Marital', 'occupation', 'relationship', 'capitalgain', 'loss', 'hoursper', 'citizen_status', 'workclass_IX', 'workclassclassVec', 'education_IX', 'educationclassVec', 'Marital_IX', 'MaritalclassVec', 'occupation_IX', 'occupationclassVec', 'relationship_IX', 'relationshipclassVec', 'citizen_status_IX', 'citizen_statusclassVec', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 2174, 0, 40, 'United-States', 4.0, [9, [4], [1.0]], 2.0, [16, [2], [1.0]], 1.0, [7, [1], [1.0]], 3.0, [15, [3], [1.0]], 1.0, [6, [1], [1.0]], 0.0, [41, [0], [1.0]], [99, [4, 11, 26, 35, 48, 53, 94, 95, 96, 98], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 77516.0, 13.0, 

In [47]:
time.sleep(2)
subscription.payload_logging.get_records_count()

39

## Fairness, drift monitoring and explanations
###  Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:

Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [48]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances".format(WOS_GUID)
MONITOR_INSTANCES_URL

'https://api.aiopenscale.test.cloud.ibm.com/openscale/68f38b69-fe5e-4bd1-9c8c-a475038661f6/v2/monitor_instances'

In [49]:
fairness_paylaod = {
    "data_mart_id": WOS_GUID,
    "monitor_definition_id": "fairness",
    "parameters": {
        "features": [
            {
                "feature": "sex",
                "majority": ["Male"],
                "minority": ["Female"]
            },
            {
                "feature": "age",
                "majority": [[26,75]],
                "minority": [[18,25]]
            }
        ],
        "favourable_class": [">50K"],
        "unfavourable_class": ["<=50K"],
        "min_records": 200
    },
    "target": {
        "target_type": "subscription",
        "target_id": subscription.uid
    },
    "thresholds": [
        {
            "metric_id": "fairness_value",
            "specific_values": [
                {
                    "applies_to": [
                        {
                            "type": "tag",
                            "value": "sex",
                            "key": "feature"
                        }
                    ],
                    "value": 80
                },
                {
                    "applies_to": [
                        {
                            "type": "tag",
                            "value": "age",
                            "key": "feature"
                        }
                    ],
                    "value": 80
                }
            ],
            "type": "lower_limit",
            "value": 80
        }
    ]
}
#print(json.dumps(fairness_paylaod, indent=2))

In [50]:
response = requests.post(MONITOR_INSTANCES_URL, json=fairness_paylaod, headers=headers, verify=False)
json_data = response.json()
print()
print(json_data)
print()


{'entity': {'data_mart_id': '68f38b69-fe5e-4bd1-9c8c-a475038661f6', 'monitor_definition_id': 'fairness', 'parameters': {'favourable_class': ['>50K'], 'features': [{'feature': 'sex', 'majority': ['Male'], 'minority': ['Female']}, {'feature': 'age', 'majority': [[26, 75]], 'minority': [[18, 25]]}], 'min_records': 200, 'unfavourable_class': ['<=50K']}, 'schedule': {'repeat_interval': 60, 'repeat_type': 'minute', 'repeat_unit': 'minute'}, 'status': {'state': 'preparing'}, 'target': {'target_id': 'e900311f-0f30-47c9-8973-19092e028555', 'target_type': 'subscription'}, 'thresholds': [{'metric_id': 'fairness_value', 'specific_values': [{'applies_to': [{'key': 'feature', 'type': 'tag', 'value': 'sex'}], 'value': 80.0}, {'applies_to': [{'key': 'feature', 'type': 'tag', 'value': 'age'}], 'value': 80.0}], 'type': 'lower_limit', 'value': 80.0}]}, 'metadata': {'created_at': '2020-07-19T11:22:22.689Z', 'created_by': 'IBMid-270007D500', 'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/e40741b27da

### Get Fairness Monitor Instance

In [51]:
MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances?target.target_id={1}&target.target_type=subscription".format(WOS_GUID, subscription.uid)
print(MONITOR_INSTANCES_URL)

response = requests.get(MONITOR_INSTANCES_URL, headers=headers)
monitor_instances = response.json()["monitor_instances"]

fairness_monitor_instance_id = None

if monitor_instances is not None:
    for monitor_instance in monitor_instances:
        if "entity" in monitor_instance and "monitor_definition_id" in monitor_instance["entity"]:
            monitor_name = monitor_instance["entity"]["monitor_definition_id"]
            if "metadata" in monitor_instance and "id" in monitor_instance["metadata"]:
                id = monitor_instance["metadata"]["id"]
                if monitor_name == "fairness":
                    fairness_monitor_instance_id = id
                    
print("Fairness monitor instance id - {0}".format(fairness_monitor_instance_id))

https://api.aiopenscale.test.cloud.ibm.com/openscale/68f38b69-fe5e-4bd1-9c8c-a475038661f6/v2/monitor_instances?target.target_id=e900311f-0f30-47c9-8973-19092e028555&target.target_type=subscription
Fairness monitor instance id - 8322faa2-b268-406e-94cf-149afa3e4fac


### Get existing fairness monitoring run id

When we configure fairness monitor, the underlying OpenScale fairness service will submit the first evaluation. For this evaluation in the below cell, we are trying to get the monitoring run it.

In [52]:
# sleep for few seconds for the fairneess monitoring check to be submitted.
time.sleep(2)

def get_monitoring_run_id(monitor_instance_id):
    monitoring_run_id = None
    MONITOR_INSTANCE_RUNS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, monitor_instance_id)
    response = requests.get(MONITOR_INSTANCE_RUNS_URL, headers=headers, verify=False)
    monitoring_runs = response.json()
    
    # there can be only one run
    if monitoring_runs is not None and "runs" in monitoring_runs:
        for run in monitoring_runs["runs"]:
            if "metadata" in run and "id" in run["metadata"]:
                monitoring_run_id = run["metadata"]["id"]
                break
    return monitoring_run_id
        
fairness_monitoring_run_id = get_monitoring_run_id(fairness_monitor_instance_id)
print("Fairness monitoring run id - {0}".format(fairness_monitoring_run_id))

Fairness monitoring run id - c6024f5a-7430-448a-b2d0-0be8c582b006


### Function to get the monitoring run details

In [53]:
def get_monitoring_run_details(monitor_instance_id, monitoring_run_id):
    
    MONITORING_RUNS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs/{2}".format(WOS_GUID, monitor_instance_id, monitoring_run_id)
    response = requests.get(MONITORING_RUNS_URL, headers=headers, verify=False)
    return response.json()

In [54]:
from datetime import datetime

def check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id):
    fairness_run_status = None
    while fairness_run_status != 'finished':
        monitoring_run_details = get_monitoring_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id)
        fairness_run_status = monitoring_run_details["entity"]["status"]["state"]
        if fairness_run_status == "error":
            print(monitoring_run_details)
            break
        if fairness_run_status != 'finished':
            print(datetime.utcnow().strftime('%H:%M:%S'), fairness_run_status)
            time.sleep(10)
    print(fairness_run_status)

check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id)

# fairness_run_details = get_monitoring_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id)
# print(json.dumps(fairness_run_details, indent = 2))

11:22:25 running
11:22:35 running
11:22:45 running
11:22:55 running
finished


In [55]:
FAIRNESS_DASHBOARD_URL = DASHBOARD_URL + "/aiopenscale/insights/{0}/fairness/age?features=fairnessv2,indirect_bias,v2transaction".format(deployment_uid)

In [56]:
from IPython.display import Markdown as md
md("#### Link to IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

#### Link to IBM Watson OpenScale Fairness Dashboard: https://aiopenscale.test.cloud.ibm.com/aiopenscale/insights/85fa8a9a-63d1-4bf2-831b-d0db19733a7f/fairness/age?features=fairnessv2,indirect_bias,v2transaction

### Run on-demand Fairness
If you would like to peform an on-demand fairness check, then we need to score a fresh set of data with meta-fields, so that they would be used for indirect bias checking. So the below two cells will score and make sure these records are reached to payload logging table.

In [57]:
payload_logging(no_of_records_to_score = 200)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/8941e786-01c8-428f-ba1a-0221993fa9a8/deployments/85fa8a9a-63d1-4bf2-831b-d0db19733a7f/online
Single record scoring result: 
 fields: ['workclass', 'fnlwgt', 'education', 'education-num', 'Marital', 'occupation', 'relationship', 'capitalgain', 'loss', 'hoursper', 'citizen_status', 'workclass_IX', 'workclassclassVec', 'education_IX', 'educationclassVec', 'Marital_IX', 'MaritalclassVec', 'occupation_IX', 'occupationclassVec', 'relationship_IX', 'relationshipclassVec', 'citizen_status_IX', 'citizen_statusclassVec', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 2174, 0, 40, 'United-States', 4.0, [9, [4], [1.0]], 2.0, [16, [2], [1.0]], 1.0, [7, [1], [1.0]], 3.0, [15, [3], [1.0]], 1.0, [6, [1], [1.0]], 0.0, [41, [0], [1.0]], [99, [4, 11, 26, 35, 48, 53, 94, 95, 96, 98], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 77516.0, 13.0, 

In [58]:
time.sleep(2)
subscription.payload_logging.get_records_count()

225

### Submit the fairness check

In [59]:
if fairness_monitor_instance_id is not None:
    MONITOR_RUN_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, fairness_monitor_instance_id)
    payload = {
        "triggered_by": "user"
    }
    print("Triggering Fairness computation with {}".format(MONITOR_RUN_URL))
    response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
    json_data = response.json()
    print()
    print(json_data)
    print()
    if "metadata" in json_data and "id" in json_data["metadata"]:
        fairness_monitoring_run_id = json_data["metadata"]["id"]
    print("Done triggering Fairness check")

Triggering Fairness computation with https://api.aiopenscale.test.cloud.ibm.com/openscale/68f38b69-fe5e-4bd1-9c8c-a475038661f6/v2/monitor_instances/8322faa2-b268-406e-94cf-149afa3e4fac/runs

{'metadata': {'created_by': 'IBMid-270007D500', 'url': '/v2/monitor_instances/8322faa2-b268-406e-94cf-149afa3e4fac/runs/59bd0481-5b0d-4381-8f3d-5eabdadd45bb', 'id': '59bd0481-5b0d-4381-8f3d-5eabdadd45bb', 'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/e40741b27da5881193d18b40e6a3078d:68f38b69-fe5e-4bd1-9c8c-a475038661f6:run:59bd0481-5b0d-4381-8f3d-5eabdadd45bb', 'created_at': '2020-07-19T11:24:03.627Z'}, 'entity': {'triggered_by': 'user', 'parameters': {'training_data_last_processed_time': '2020-07-19 11:23:02.151441', 'training_data_metrics': [{'minority': {'total_fav_percent': 10.9, 'values': [{'fairness_value': 35.8, 'fav_class_percent': 10.9, 'value': 'Female'}]}, 'feature': 'sex', 'fairness_value': 35.8, 'majority': {'total_fav_percent': 30.599999999999998, 'values': [{'fav_class_percent

### Check for its status

In [60]:
check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id)

11:24:09 running
11:24:19 running
finished


In [61]:
from IPython.display import Markdown as md
md("#### To view the latest evaluation of the fairness check, please visit IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

#### To view the latest evaluation of the fairness check, please visit IBM Watson OpenScale Fairness Dashboard: https://aiopenscale.test.cloud.ibm.com/aiopenscale/insights/85fa8a9a-63d1-4bf2-831b-d0db19733a7f/fairness/age?features=fairnessv2,indirect_bias,v2transaction

## Additional data to help debugging

In [62]:
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("OpenScale Datamart id: {}".format(WOS_GUID))
print("OpenScale Subscription id: {}".format(subscription.uid))
print("OpenScale Fairness Monitor Instance id: {}".format(fairness_monitor_instance_id))
print("OpenScale Fairness Monitoring Run id: {}".format(fairness_monitoring_run_id))


Model id: 1f426b8c-1a05-47c6-97ef-3c45d9e460a7
Deployment id: 85fa8a9a-63d1-4bf2-831b-d0db19733a7f
OpenScale Datamart id: 68f38b69-fe5e-4bd1-9c8c-a475038661f6
OpenScale Subscription id: e900311f-0f30-47c9-8973-19092e028555
OpenScale Fairness Monitor Instance id: 8322faa2-b268-406e-94cf-149afa3e4fac
OpenScale Fairness Monitoring Run id: 59bd0481-5b0d-4381-8f3d-5eabdadd45bb
